### deribit options liquidity profile
   * on day t, what are the options available
   * on day t, what are the available options liquidiy, trade volume, spread, orderbook updates, across different strikes, put/call
   * spread information about options

In [17]:
from cdcqr.data.dataloader import data_loader
from cdcqr.data.deribit.data_utils import DeribitUtils as derU
from datetime import datetime, timedelta
import calendar
import pandas as pd
from importlib import reload
import matplotlib.pyplot as plt
import numpy as np
from cdcqr.common.config import LOCAL_DATA_DIR
import os

In [ ]:
exchange = 'deribit'
date = '2021/09/22'
symbol = 'OPTIONS'
data_type = 'trades'
opt_trades = data_loader(exchange, date,data_type, symbol)

In [ ]:
opt_trades.head()

In [ ]:
opt_trades_parsed = opt_trades.pipe(derU.parse_optSymbol_col).pipe(derU.parse_time_col)

In [ ]:
coin = 'BTC'
opt_trades_i = opt_trades_parsed.query('instrument==@coin')

In [ ]:
df = opt_trades_i.groupby('expire')['amount'].sum().sort_values().to_frame().reset_index()

In [ ]:
df['date'] = date

### function test of important expiry dates

In [ ]:
## get important expiry dates
dt = datetime(2021,12,1)
display(derU.DeribitUtils.next_friday(dt))
display(derU.DeribitUtils.last_friday_of_the_month(dt))
display(derU.DeribitUtils.last_friday_of_next_month(dt))
display(derU.DeribitUtils.last_friday_of_the_quarter(dt))
display(derU.DeribitUtils.last_friday_of_the_2nd_to_next_quarter(dt))
display(derU.DeribitUtils.last_friday_of_the_3rd_to_next_quarter(dt))

In [ ]:
derU.DeribitUtils.get_important_option_expire_dates(datetime(2021,5,19))

#### daily trading volume

In [ ]:
time_index = pd.date_range(datetime(2021,1,1), datetime(2021,10,31))
dfs = []
coin = 'BTC'
exchange = 'deribit'
symbol = 'OPTIONS'
data_type = 'trades'
for date in time_index:
    opt_trades = data_loader(exchange, date, data_type, symbol)
    opt_trades_parsed = opt_trades.pipe(derU.parse_optSymbol_col).pipe(derU.parse_time_col)
    opt_trades_i = opt_trades_parsed.query('instrument==@coin')
    df = opt_trades_i.groupby(['side','type'])['amount'].sum().reset_index()
    df['weekday'] = date.weekday()
    dfs.append(df)


In [ ]:
df1 = pd.concat(dfs)

In [ ]:
df2 = df1.groupby(['side','type','weekday'])['amount'].sum().reset_index()

In [ ]:
fig = px.bar(df2, x="weekday", y="amount", color="type",
             pattern_shape="side", pattern_shape_sequence=["x", "."])
fig.show()

In [ ]:
df = pd.DataFrame(date2vol, index=[0]).T.reset_index()
df

In [ ]:
df['weekday'] = df['index'].dt.weekday

In [ ]:
df.groupby('weekday').mean().plot()
plt.ylim((0,30000))
plt.legend(['avg_btc_traded'])
plt.title('avg_btc_traded 2021 Jan-Oct')

#### option availability across weekdays

In [ ]:
time_index = pd.date_range(datetime(2021,6,7), datetime(2021,6,13))

In [ ]:
dfs = []
coin = 'BTC'
exchange = 'deribit'
symbol = 'OPTIONS'
data_type = 'trades'
for date in time_index:
    opt_trades = data_loader(exchange, date,data_type, symbol)
    opt_trades_parsed = opt_trades.pipe(derU.parse_optSymbol_col).pipe(derU.parse_time_col)
    opt_trades_i = opt_trades_parsed.query('instrument==@coin')
    df = opt_trades_i.groupby('expire')['amount'].sum().sort_values().to_frame().reset_index()
    df['date'] = date
    dfs.append(df)

In [ ]:
df1 = pd.concat(dfs).set_index(['date','expire']).unstack()

In [ ]:
df1.columns = df1.columns.droplevel(0)

In [ ]:
df1.T

#### total trade volume across all strikes

In [ ]:
dfs = []
coin = 'BTC'
exchange = 'deribit'
symbol = 'OPTIONS'
data_type = 'trades'
for date in time_index:
    opt_trades = data_loader(exchange, date,data_type, symbol)
    opt_trades_parsed = opt_trades.pipe(derU.parse_optSymbol_col).pipe(derU.parse_time_col)
    opt_trades_i = opt_trades_parsed.query('instrument==@coin')
    df = opt_trades_i.groupby(['strike','type'])['amount'].sum().reset_index()
    df['date'] = date
    df['log(K)'] = np.log(df['strike']) 
    dfs.append(df)

In [ ]:
df1 = pd.concat(dfs)
df1.head()

In [ ]:
fig = px.scatter(df1, x = 'date', y='strike' , size="amount", color='type',range_y=[0,220000], width=800, height=1200)
fig.show()

In [ ]:
opt_trades_i

#### spread

In [2]:
date_range = pd.date_range(datetime(2021,1,1), datetime(2021,10,31))

In [3]:
exchange = 'deribit'

In [4]:
data_type= 'quotes'
date = date_range[0]
symbol='OPTIONS'

In [5]:
def date2quote_spread(date):
    opt_quote = data_loader(exchange, date, data_type, symbol)

    liq = opt_quote.groupby('symbol').count()['exchange'].sort_values(ascending=False)

    liq_sym = liq[liq.index.str.contains('BTC')].index[0]

    opt_quote_l = opt_quote.query('symbol==@liq_sym')

    opt_quote_l = opt_quote_l[['symbol','timestamp','ask_price','bid_price']]

    opt_quote_l['spread'] = opt_quote_l['ask_price']-opt_quote_l['bid_price']

    opt_quote_l['mid'] = 0.5*(opt_quote_l['ask_price']+opt_quote_l['bid_price'])

    opt_quote_l['spreadr'] = opt_quote_l['spread']/opt_quote_l['mid']

    opt_quote_l1 = opt_quote_l.pipe(derU.parse_time_col).set_index('timestamp_dt').resample('1Min').last().iloc[1:,:].drop(columns=['timestamp'])

    return opt_quote_l1

In [ ]:
ls = []
for date in date_range:
    print(date)
    ls.append(date2quote_spread(date))

In [1]:
df_res

NameError: name 'df_res' is not defined

In [10]:
df_res = pd.concat(ls)

In [23]:
df_res.describe()

,ask_price,bid_price,spread,mid,spreadr
count,385293.000000,392621.000000,383568.000000,383568.000000,383568.000000
mean,0.327633,0.303855,0.019695,0.319129,0.114927
std,0.761297,0.709166,0.077622,0.739460,0.245257
min,0.000500,0.000500,0.000500,0.000750,0.001059
25%,0.033000,0.031000,0.001500,0.032500,0.027027
50%,0.087000,0.082500,0.003000,0.085500,0.042553
75%,0.384000,0.342500,0.010500,0.372000,0.068966
max,5.750000,5.386000,5.724500,5.559500,1.999651


In [22]:
df_res.quantile(0.99)

ask_price    5.361
bid_price    4.968
spread       0.369
mid          5.169
spreadr      1.000
Name: 0.99, dtype: float64

In [21]:
df_res.quantile(0.01)

ask_price    0.001500
bid_price    0.000500
spread       0.000500
mid          0.001000
spreadr      0.007905
Name: 0.01, dtype: float64

In [18]:
df_res.to_csv(os.path.join(LOCAL_DATA_DIR, 'quote_spread.csv'))